# Tutorial 4: Attaching a Dataset

In [1]:
import libspn as spn
import tensorflow as tf

### Building a Test Graph with Random Weights

In [2]:
iv_x = spn.IVs(num_vars=2, num_vals=2, name="iv_x")
sum_11 = spn.Sum((iv_x, [0,1]), name="sum_11")
sum_12 = spn.Sum((iv_x, [0,1]), name="sum_12")
sum_21 = spn.Sum((iv_x, [2,3]), name="sum_21")
sum_22 = spn.Sum((iv_x, [2,3]), name="sum_22")
prod_1 = spn.Product(sum_11, sum_21, name="prod_1")
prod_2 = spn.Product(sum_11, sum_22, name="prod_2")
prod_3 = spn.Product(sum_12, sum_22, name="prod_3")
root = spn.Sum(prod_1, prod_2, prod_3, name="root")
iv_y = root.generate_ivs(name="iv_y")
spn.generate_weights(root, init_value=spn.ValueType.RANDOM_UNIFORM(0, 1))

### Visualizing the SPN Graph

In [3]:
spn.display_spn_graph(root)

### Specify Training Data

In [4]:
dataset=spn.CSVFileDataset('data.csv', num_vals=[2, 2], defaults=[[-1],[-1],[-1]],
                           num_labels=1, num_epochs=10, batch_size=10, shuffle=False)
samples, labels = dataset.get_data()
iv_x.attach_feed(samples)
iv_y.attach_feed(labels)

[INFO] [spn.Dataset:get_data] Building dataset operations


### Add Learning Ops

In [5]:
init_weights = spn.initialize_weights(root)
learning = spn.EMLearning(root, initial_accum_value=2)
init_learning = learning.reset_accumulators()
accumulate_updates = learning.accumulate_updates()
update_spn = learning.update_spn()
likelihood = tf.reduce_mean(learning.value.values[root])

### Run Learning

In [6]:
epoch = 0
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    try:    
        while run():
            likelihood_arr, _ = sess.run([likelihood, accumulate_updates])
            print("Avg. Likelihood: %s" % (likelihood_arr))
            sess.run(update_spn)
            
    except tf.errors.OutOfRangeError:
        print("Done!")

Avg. Likelihood: -1.42971
Avg. Likelihood: -1.29909
Avg. Likelihood: -1.28976
Avg. Likelihood: -1.28698
Avg. Likelihood: -1.28579
Avg. Likelihood: -1.28517
Avg. Likelihood: -1.28481
Avg. Likelihood: -1.28458
Avg. Likelihood: -1.28443
Avg. Likelihood: -1.28432
Done!


In [7]:
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    try:
        while run():
            likelihoods, _ =  sess.run([likelihood, accumulate_updates])
            print("Avg. Likelihood: %s" % (avg_likelihood))
            sess.run(update_spn)
            
    except tf.errors.OutOfRangeError:
        print("TRAINING DONE!")

NameError: name 'avg_likelihood' is not defined